# Evaluation all Models on Open Data

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from scipy.stats import norm

In [3]:
def aps_ci_logit(y_true, y_scores, epsilon = 0.05):
    aps = average_precision_score(y_true, y_scores)
    n = len(y_true)
    
    nu = np.log(aps/(1-aps))
    se = ((n*aps*(1-aps))**((-1)*1/2))
    norm_se = norm.ppf(1-epsilon/2)*se
    
    exp_lower = np.exp(nu-norm_se)
    exp_upper = np.exp(nu+norm_se)
    
    lower = exp_lower/(1+exp_lower)
    upper = exp_upper/(1+exp_upper)
    
    return (lower, upper)

In [4]:
import auc_delong_xu

In [5]:
from auc_delong_xu import auc_ci_Delong

#auc, auc_var, ci = auc_ci_Delong(y_true=y_true, y_scores=y_score)

#print('ROC AUC: %s, Conf.' % auc) 
#print('Confidence Interval: %s (95%% confidence)' % str(ci))

In [6]:
import os.path, time
#print("last modified: %s" % time.ctime(os.path.getmtime(file)))
#print("created: %s" % time.ctime(os.path.getctime(file)))

In [7]:
from os import listdir
from os.path import isfile, join

In [8]:
dd = pd.DataFrame(columns = ['full_name', 'name', 'type', 'ctime_d', 'ctime', 'time_num'])

In [9]:
i = 0
for f in listdir('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp'): 

#listdir('C:/Users/asmirnova5/Desktop/Python-scripts/Python-notebook/depp/model_step1'):


    if len( f.split('.')) > 1:
        tt =  f.split('.')[len( f.split('.')) - 1]
        tt0 = f.split('.' + tt)[0]
    else:
        tt = f.split('.')[0]
        tt0 = f
    ct_n = os.path.getctime('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp' + '/' + f)
    ct = datetime.strftime(datetime.fromtimestamp(ct_n), "%d.%m.%Y %H:%M:%S")
    #k = [f, tt, ct_n, ct]
    t_n = int(datetime.fromtimestamp(ct_n).year*10000 + datetime.fromtimestamp(ct_n).month*100 + datetime.fromtimestamp(ct_n).day)
    dd = dd.append({'full_name': f, 'name': tt0, 'type':tt, 'ctime_d': ct_n, 'ctime': ct, 'time_num':t_n}, ignore_index=True)
    #print(f, )

In [10]:
#d1 = dd[((dd['type'] == 'h5')) & (dd['time_num'] >= 20200908)]
d1 = dd[((dd['type'] == 'h5'))]

In [11]:
#d2 = dd[(dd['type'] == 'csv')  & (dd['time_num'] >= 20200908)][['full_name', 'name' ]]
d2 = dd[(dd['type'] == 'csv')][['full_name', 'name' ]]
d2.columns = ['csv_name', 'name' ]

In [12]:
dd_all = pd.merge(d1, d2, how='left', on='name')

In [13]:
dd_all.shape

(217, 7)

In [50]:
best_model = ['model_spider6_china_grid_0.0002_7_15_30_0.0002_200420_15',
             'model_spider6_china_grid_0.0002_5_15_30_0.0002_200419_0',
             'model_spider8_china_grid_15_5_15_60_0.0002_200429_3',
             'model_spider8_china_grid_20_5_15_100_0.0002_200503_18',
             'model_cnn_china_grid_False_5_15_60_3_200428_18',
             'model_cnn_china_grid_False_5_15_60_6_200429_19',
             'model_cnn_china_grid_False_5_5_100_6_200428_12',
             'model_cnn_china_grid_False_5_10_30_8_200501_0']

In [14]:
best_model = [
    'model_spider6_china_grid_0.0002_7_15_30_0.0002_200420_15' ,
'model_spider6_grid_wo_dropconv_0.25_0tTrue_0.0001_3_10_100' ,
'model_spider6_grid_wo_dropconv_None_NonetTrue_0.0001_3_10_100', 

'model_spider8_china_grid_15_5_15_60_0.0002_200429_3' , 
'model_spider8_internal_0.25_0t15_5_15_60_0.0002_200610_10' , 
'model_spider8_internal_None_Nonet15_5_15_60_0.0002_200608_17'

]

In [15]:
len(set(best_model))

6

In [16]:
dd_best = dd_all[dd_all['name'].isin(best_model)]

In [17]:
dd_best.shape

(6, 7)

In [22]:
dd_best

,full_name,name,type,ctime_d,ctime,time_num,csv_name
112,model_spider6_china_grid_0.0002_7_15_30_0.0002...,model_spider6_china_grid_0.0002_7_15_30_0.0002...,h5,1.618921e+09,20.04.2021 15:16:25,20210420,model_spider6_china_grid_0.0002_7_15_30_0.0002...
168,model_spider6_grid_wo_dropconv_0.25_0tTrue_0.0...,model_spider6_grid_wo_dropconv_0.25_0tTrue_0.0...,h5,1.623089e+09,07.06.2021 21:06:47,20210607,model_spider6_grid_wo_dropconv_0.25_0tTrue_0.0...
169,model_spider6_grid_wo_dropconv_None_NonetTrue_...,model_spider6_grid_wo_dropconv_None_NonetTrue_...,h5,1.623068e+09,07.06.2021 15:18:32,20210607,model_spider6_grid_wo_dropconv_None_NonetTrue_...
174,model_spider8_china_grid_15_5_15_60_0.0002_200...,model_spider8_china_grid_15_5_15_60_0.0002_200...,h5,1.619658e+09,29.04.2021 03:52:07,20210429,model_spider8_china_grid_15_5_15_60_0.0002_200...
185,model_spider8_internal_0.25_0t15_5_15_60_0.000...,model_spider8_internal_0.25_0t15_5_15_60_0.000...,h5,1.623311e+09,10.06.2021 10:36:27,20210610,model_spider8_internal_0.25_0t15_5_15_60_0.000...
202,model_spider8_internal_None_Nonet15_5_15_60_0....,model_spider8_internal_None_Nonet15_5_15_60_0....,h5,1.623163e+09,08.06.2021 17:29:22,20210608,model_spider8_internal_None_Nonet15_5_15_60_0....


In [18]:
dd = pd.DataFrame(columns = ['full_name', 'name', 'type', 'ctime_d', 'ctime', 'time_num'])

In [19]:
i = 0
for f in listdir('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/best_dense/Open'): 

    if len( f.split('.')) > 1:
        tt =  f.split('.')[len( f.split('.')) - 1]
        tt0 = f.split('.' + tt)[0]
    else:
        tt = f.split('.')[0]
        tt0 = f
    ct_n = os.path.getctime('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/best_dense/Open' + '/' + f)
    ct = datetime.strftime(datetime.fromtimestamp(ct_n), "%d.%m.%Y %H:%M:%S")
    #k = [f, tt, ct_n, ct]
    t_n = int(datetime.fromtimestamp(ct_n).year*10000 + datetime.fromtimestamp(ct_n).month*100 + datetime.fromtimestamp(ct_n).day)
    dd = dd.append({'full_name': f, 'name': tt0, 'type':tt, 'ctime_d': ct_n, 'ctime': ct, 'time_num':t_n}, ignore_index=True)
    #print(f, )

In [20]:
d1 = dd[((dd['type'] == 'h5'))]
d2 = dd[(dd['type'] == 'csv')][['full_name', 'name' ]]
d2.columns = ['csv_name', 'name' ]
dd_all = pd.merge(d1, d2, how='left', on='name')

In [21]:
dd_all

,full_name,name,type,ctime_d,ctime,time_num,csv_name
0,model_dense_OpenData_c3c3_k10_t0.5_w3.h5,model_dense_OpenData_c3c3_k10_t0.5_w3,h5,1.620732e+09,11.05.2021 14:14:51,20210511,model_dense_OpenData_c3c3_k10_t0.5_w3.csv
1,model_dense_OpenData_c4c4_k10_t0.5_w5.h5,model_dense_OpenData_c4c4_k10_t0.5_w5,h5,1.620732e+09,11.05.2021 14:14:51,20210511,model_dense_OpenData_c4c4_k10_t0.5_w5.csv
2,model_fDenseNet6_china_grid_0.0002_5_15_100_0....,model_fDenseNet6_china_grid_0.0002_5_15_100_0....,h5,1.620732e+09,11.05.2021 14:14:51,20210511,model_fDenseNet6_china_grid_0.0002_5_15_100_0....
3,model_fDenseNet8_china_grid_0.0002_7_15_100_0....,model_fDenseNet8_china_grid_0.0002_7_15_100_0....,h5,1.620732e+09,11.05.2021 14:14:52,20210511,model_fDenseNet8_china_grid_0.0002_7_15_100_0....


In [58]:
dd_best = pd.concat([dd_all, dd_best], axis = 0)

In [59]:
dd_best.shape

(12, 7)

In [23]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# ***

In [24]:
target = 'LABEL'
index_month = 'MONTH'
list_of_vars_for_strat = ['MONTH']
sort_by_var = 'ID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

In [25]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_chinese/'
train_for = pd.read_csv(PATH + 'china_train_128_prep.csv')
valid_for = pd.read_csv(PATH + 'china_val_128_prep.csv')
test_for = pd.read_csv(PATH + 'china_test_128_prep.csv')

In [26]:
y_train = train_for[target]
y_val = valid_for[target]
y_test = test_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [27]:
X_1_2.shape, X_2_2.shape, X_3_2.shape

((792004, 128), (99001, 128), (99001, 128))

# ***

In [28]:
dd_best['Val_APS'] = None
dd_best['Val_APS_conf_l'] = None
dd_best['Val_APS_conf_r'] = None
dd_best['Val_GINI'] = None
dd_best['Val_AUC'] = None
dd_best['Val_AUC_conf_l'] = None
dd_best['Val_AUC_conf_r'] = None

dd_best['TEST_APS'] = None
dd_best['TEST_APS_conf_l'] = None
dd_best['TEST_APS_conf_r'] = None
dd_best['TEST_GINI'] = None
dd_best['TEST_AUC'] = None
dd_best['TEST_AUC_conf_l'] = None
dd_best['TEST_AUC_conf_r'] = None

dd_best['Count_epoch'] = None
dd_best['Count_feature'] = None
dd_best['Min_loss'] = None

In [29]:
dd_best['LR_schedule'] = None

dd_best['saved_money_test'] = None
dd_best['saved_money_val'] = None
dd_best['saved_money_oot'] = None

In [30]:
dd_best['Val_APS_conf_int'] = None
dd_best['TEST_APS_conf_int'] = None
dd_best['OOT2013_APS_conf_int'] = None

dd_best['TEST_AUC_midle'] = None
dd_best['TEST_AUC_conf_int'] = None

In [31]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Activation, Conv1D, Concatenate, AveragePooling1D
import tensorflow

In [32]:
""" 
blocks/one_d.py
Author: Ankit Gupta

Implementations of various DenseNet blocks for 1D sequences

This module contains helper functions that define the various subcomponents of a DenseNet. This includes dense blocks and transition blocks.

"""

#from tf.keras.layers import BatchNormalization, Activation, Conv1D, Concatenate, AveragePooling1D


def H_l(k, bottleneck_size, kernel_width):
    """ 
    A single convolutional "layer" as defined by Huang et al. Defined as H_l in the original paper
    
    :param k: int representing the "growth rate" of the DenseNet
    :param bottleneck_size: int representing the size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :param kernel_width: int representing the width of the main convolutional kernel
    :return a function wrapping the keras layers for H_l
    """

    use_bottleneck = bottleneck_size > 0
    num_bottleneck_output_filters = k * bottleneck_size

    def f(x):
        if use_bottleneck:
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = Conv1D(
                num_bottleneck_output_filters,
                1,
                strides=1,
                padding="same",
                dilation_rate=1)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv1D(
            k,
            kernel_width,
            strides=1,
            padding="same",
            dilation_rate=1)(x)
        return x
    return f


def dense_block(k, num_layers, kernel_width, bottleneck_size):
    """
    A single dense block of the DenseNet
    
    :param k: int representing the "growth rate" of the DenseNet
    :param num_layers: int represending the number of layers in the block
    :param kernel_width: int representing the width of the main convolutional kernel
    :param bottleneck_size: int representing the size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :return a function wrapping the entire dense block
    """
    def f(x):
        layers_to_concat = [x]
        for _ in range(num_layers):
            x = H_l(k, bottleneck_size, kernel_width)(x)
            layers_to_concat.append(x)
            x = Concatenate(axis=-1)(layers_to_concat)
        return x
    return f


def transition_block(pool_size=2, stride=2, theta=0.5):
    """
    A single transition block of the DenseNet
    
    :param pool_size: int represending the width of the average pool
    :param stride: int represending the stride of the average pool
    :param theta: int representing the amount of compression in the 1x1 convolution. Set to 1 for no compression.
    :return a function wrapping the entire transition block
    """    
    assert theta > 0 and theta <= 1

    def f(x):
        num_transition_output_filters = int(int(x.shape[2]) * float(theta))
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv1D(
            num_transition_output_filters,
            1,
            strides=1,
            padding="same",
            dilation_rate=1)(x)
        x = AveragePooling1D(
            pool_size=pool_size,
            strides=stride,
            padding="same")(x)
        return x
    return f


def DenseNet_1(
        k,
        block_sizes,
        conv_kernel_width,
        bottleneck_size,
        transition_pool_size,
        transition_pool_stride,
        theta,
        initial_conv_width,
        initial_stride,
        initial_filters,
        initial_pool_width,
        initial_pool_stride,
        use_global_pooling):
    def f(x):
        x = Conv1D(
            initial_filters,
            initial_conv_width,
            strides=initial_stride,
            padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = MaxPooling1D(
            pool_size=initial_pool_width,
            strides=initial_pool_stride,
            padding="same")(x)

        # Add all but the last dense block
        for block_size in block_sizes[:-1]:
            x = dense_block(
                k,
                block_size,
                conv_kernel_width,
                bottleneck_size)(x)
            x = transition_block(
                pool_size=transition_pool_size,
                stride=transition_pool_stride,
                theta=theta)(x)

        # Add the last dense block
        final_block_size = block_sizes[-1]
        x = dense_block(
            k,
            final_block_size,
            conv_kernel_width,
            bottleneck_size)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        if use_global_pooling:
            x = GlobalAveragePooling1D()(x)
        return x
    return f



class DenseNetCustom(tensorflow.keras.models.Model):
    """  
    Create a Keras Model Object that is an implementation of DenseNet with a custom number of parameters. The number of layers 
    per dense block can be specified by block_sizes.
    :param input_shape: The shape of the inputs without the batch dimension. This should be a valid 1D sequence, 
    such as (244, 25). 
    :param num_outputs: the number of classes to predict
    :param k: The "growth rate" of the DenseNet model
    :param block_sizes: A list of ints with the number of layers in each block. Example: [5, 10, 25, 17].
    :param conv_kernel_width: The kernel width of each convolution in the dense blocks.
    :param bottleneck_size: The size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :param transition_pool_size: pool_size in the transition layer
    :param transition_pool_stride: pooling stride in the transition layer
    :param theta: Amount of compression in the transition layer. Set to 1 for no compression.
    :param initial_conv_width: Kernel width for the one convolution before the dense blocks
    :param initial_stride: Stride for the one convolution before the dense blocks
    :param initial_filters: Number of filters for the one convolution before the dense blocks
    :param initial_pool_width: pool_size for the one pooling before the dense blocks
    :param initial_pool_stride: stride for the one pooling before the dense blocks 
    """
    def __init__(
            self,
            input_shape,
            num_outputs=1000,
            k=32,
            block_sizes=None,
            conv_kernel_width=3,
            bottleneck_size=4,
            transition_pool_size=2,
            transition_pool_stride=2,
            theta=0.5,
            initial_conv_width=7,
            initial_stride=2,
            initial_filters=64,
            initial_pool_width=3,
            initial_pool_stride=2):
        if not block_sizes:
            raise ValueError("block_sizes must be specified")
        model_input = Input(shape=input_shape)
        output = DenseNet_1(
            k,
            block_sizes,
            conv_kernel_width,
            bottleneck_size,
            transition_pool_size,
            transition_pool_stride,
            theta,
            initial_conv_width,
            initial_stride,
            initial_filters,
            initial_pool_width,
            initial_pool_stride,
            use_global_pooling=True)(model_input)
        output = Dense(num_outputs, activation="softmax")(output)
        super(DenseNetCustom, self).__init__(model_input, output)

In [33]:
dd_all = dd_best

In [34]:
dd_all.shape

(6, 33)

In [35]:
#бизнес-метрика: на каждом периоде отбираем ТОП-ТТ и считаем по ним убыток
#по данным из AF, в неделю проверяется 40 ТТ
#поэтому для oot за 2013г берется 40, для test/ val берется 5, т.к. test/ val - это пот 10% от выборки
#scores_optuna['saved_money_test_multiply'] = [saved_money_test_only_1*5] - 5 - это коэффициент, 
#который приводит результат к полным данным, для test/ val нужно брать 10


for j in dd_all.index:
    
    
    mod = dd_all['full_name'][j]
    tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
    print (j, mod, 'begin', '| time: ' , tm)
    
    if mod.find('_dense_') >= 0:
        model_X = tf.keras.models.load_model(('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/' +mod),
                                             custom_objects={'DenseNetCustom':DenseNetCustom})
    else:
        model_X = load_model('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/' +mod)
    
    if str(dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0]) != 'nan':
        history_XX = pd.read_csv('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/' + 
                                 dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0])
    
    test = pd.DataFrame.from_dict(model_X.get_config()['layers'])
    
    
    #"""
    if test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1] == 128:
        
        X_2 = X_2_2.copy(deep=True)   
        X_2['predict_class'] = model_X.predict(X_2_2)[:,1]

        dd_all.at[j, 'Val_APS'] = metrics.average_precision_score(y_val, X_2['predict_class'])
        dd_all.at[j, 'Val_GINI']  = 2*metrics.roc_auc_score(y_val, X_2['predict_class']) - 1
        
        
        #####
        ci_a = aps_ci_logit(y_val, X_2['predict_class'])
        dd_all.at[j,'Val_APS_conf_l'] = ci_a[0]
        dd_all.at[j,'Val_APS_conf_r'] = ci_a[1]
        dd_all.at[j,'Val_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        auc, auc_var, ci = auc_ci_Delong(y_true=y_val, y_scores=X_2['predict_class'])
        dd_all.at[j,'Val_AUC'] = auc
        dd_all.at[j,'Val_AUC_conf_l'] = ci[0]
        dd_all.at[j,'Val_AUC_conf_r'] = ci[1]
        dd_all.at[j,'Val_AUC_conf_int'] = (ci[1] - ci[0])/2
        del X_2

        ####### test
        
        X_3 = X_3_2.copy(deep=True)   
        X_3['predict_class'] = model_X.predict(X_3_2)[:,1]
        
        dd_all.at[j, 'TEST_APS'] = metrics.average_precision_score(y_test, X_3['predict_class'])
        dd_all.at[j, 'TEST_GINI']  = 2*metrics.roc_auc_score(y_test, X_3['predict_class']) - 1
        
        ci_a = aps_ci_logit(y_test, X_3['predict_class'])
        dd_all.at[j,'TEST_APS_conf_l'] = ci_a[0]
        dd_all.at[j,'TEST_APS_conf_r'] = ci_a[1]
        
        dd_all.at[j,'TEST_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        auc, auc_var, ci = auc_ci_Delong(y_true=y_test, y_scores=X_3['predict_class'])
        dd_all.at[j,'TEST_AUC'] = auc
        dd_all.at[j,'TEST_AUC_conf_l'] = ci[0]
        dd_all.at[j,'TEST_AUC_conf_r'] = ci[1]
        dd_all.at[j,'TEST_AUC_conf_int'] = (ci[1] - ci[0])/2
        del X_3
        
        

    ####
    dd_all.at[j, 'Count_epoch'] = history_XX.shape[0]
    dd_all.at[j, 'Min_loss'] = min(history_XX['val_loss'])
    dd_all.at[j, 'Count_feature'] = test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1]
    
    if 'lr' in history_XX.columns:
        dd_all.at[j, 'LR_schedule'] = len(np.unique(history_XX['lr']))
        
    dd_all.to_excel('BEST_china_conf_int_20210611.xlsx')
                                                                        
    #tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
    #print(j, mod, 'end', '| time: ' , tm)

112 model_spider6_china_grid_0.0002_7_15_30_0.0002_200420_15.h5 begin | time:  11.06.2021 11:54:28
168 model_spider6_grid_wo_dropconv_0.25_0tTrue_0.0001_3_10_100.h5 begin | time:  11.06.2021 11:55:25
169 model_spider6_grid_wo_dropconv_None_NonetTrue_0.0001_3_10_100.h5 begin | time:  11.06.2021 11:55:47
174 model_spider8_china_grid_15_5_15_60_0.0002_200429_3.h5 begin | time:  11.06.2021 11:56:11
185 model_spider8_internal_0.25_0t15_5_15_60_0.0002_200610_10.h5 begin | time:  11.06.2021 11:57:37
202 model_spider8_internal_None_Nonet15_5_15_60_0.0002_200608_17.h5 begin | time:  11.06.2021 11:59:06
